    !pip install option-price

In [1]:
from optionprice import Option
import numpy as np
from scipy.stats import norm
import datetime

In [2]:
call_option_pricing = Option(european=True,
                    kind='call',
                    s0=100,
                    k=120,
                    t=45,
                    sigma=0.01,
                    r=0.05,
                    dv=0)

In [3]:
call_price = call_option_pricing.getPrice()
call_price

0.0

In [4]:
put_option_pricing = Option(european=True,
                    kind='put',
                    s0=100,
                    k=120,
                    t=45,
                    sigma=0.01,
                    r=0.05,
                    dv=0)

In [5]:
print(put_option_pricing)

Type:           European
Kind:           put
Price initial:  100
Price strike:   120
Volatility:     1.0%
Risk free rate: 5.0%
Start Date:     2020-08-26
Expire Date:    2020-10-10
Time span:      45.0 days


In [6]:
put_price = put_option_pricing.getPrice()
put_price

19.26254927241017

In [7]:
s0,k,t,sigma,r=100,120,45/365,0.01,0.05

In [8]:
def pricing_EU_call(s0, k, t,r, sigma):
    sqrtT = np.sqrt(t)
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma **2) * t) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT

    expRT = np.exp(- r * t)

    callResult =(s0 * norm.cdf(d1) - k *expRT * norm.cdf(d2))
    return callResult

In [9]:
pricing_EU_call(s0, k, t,r, sigma)

0.0

In [10]:
def pricing_EU_put(s0, k, t,r, sigma):

    sqrtT = np.sqrt(t)
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma **2) * t) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT

    expRT = np.exp(- r * t)
    
    putResult = (k * expRT * norm.cdf(-d2, 0.0, 1.0) - s0 * norm.cdf(-d1, 0.0, 1.0))
    return putResult

In [11]:
pricing_EU_put(s0, k, t,r, sigma)

19.26254927241017

## Option Pricing with Monte Carlo

In [12]:
T = (datetime.date(2013,9,21) - datetime.date(2013,9,3)).days / 365.0

In [13]:
discount_factor = np.exp(-r * T)

In [14]:
def sim_asset_price(S,v,r,T):
    return S * np.exp((r - 0.5 * v**2) * T + v * np.sqrt(T) * np.random.normal(0,1.0))

In [15]:
def call_payoff(S_T, K):
    return max(S_T - K, 0.0)

In [16]:
S = 100
v = 0.2
r = 0.003 
K= 110

In [17]:
simulations = 90000
payoffs = []
for i in range(simulations):
    S_T = sim_asset_price(S,v,r,T)
    payoffs.append(call_payoff(S_T, K))

In [18]:
call_price = discount_factor * (sum(payoffs) / float(simulations))
print('Call Price: {}'.format(call_price))

Call Price: 0.027695167572156624


In [19]:
call_option = Option(european=True,
                    kind='call',
                    s0=100,
                    k=110,
                    t=18,
                    sigma=0.2,
                    r=0.003,
                    dv=0)

In [20]:
call_price = call_option.getPrice(method='MC',iteration = 90000)
print('Call price is {}'.format(call_price))

Call price is 0.026409308624665388


In [21]:
def put_payoff(S_T, K):
    return max(K-S_T, 0.0)

In [22]:
simulations = 90000
payoffs = []
for i in range(simulations):
    S_T = sim_asset_price(S,v,r,T)
    payoffs.append(put_payoff(S_T, K))

In [23]:
put_price = discount_factor * (sum(payoffs) / float(simulations))
print('Put Price is {}'.format(put_price))

Put Price is 10.008049557622066


In [24]:
put_price = Option(european=True,
                    kind='put',
                    s0=100,
                    k=110,
                    t=18,
                    sigma=0.2,
                    r=0.003,
                    dv=0)

In [25]:
put_price = put_price.getPrice(method='MC',iteration = 90000)
print('Put price is {}'.format(put_price))

Put price is 10.002323029653812
